In [1]:
import serial
import time
import numpy as np
from itertools import product
import random
import pandas as pd
import os

In [2]:
class Motionstim8:

    def __init__(self, freq=22):

        # The Motionstim8 device has a serial port
        self.serialPort = serial.Serial('COM3') #COM5 on Misha's laptop

        # The default main stimulation frequency being used is 20 Hz
        self.stimulationFrequency = freq

        # The default group stimulation frequency being used is 50 Hz
        self.groupStimulationFrequency = 1

        # Set the N factor (0 by default)
        self.nFactor = 0

        # The number of channels in the FES device
        self.nChannels = 8

        # The device channel modes (singlets)
        self.pulseModes = [0] * self.nChannels

        # The device pulse widths
        self.pulseWidths = [100] * self.nChannels

        # The device amplitudes
        self.amplitudes = [0] * self.nChannels




    def OpenSerialPort(self, comPortName):
        # configure the serial connections (the parameters differs on the device you are connecting to)
        self.serialPort.port = comPortName
        self.serialPort.baudrate = 115200
        self.serialPort.parity = serial.PARITY_NONE
        self.serialPort.stopbits = serial.STOPBITS_ONE
        self.serialPort.bytesize = serial.EIGHTBITS
        self.serialPort.timeout = 500
        self.serialPort.write_timeout = 500
        self.serialPort.xonxoff  = False
        self.serialPort.rtscts = False
        self.dsrdtr = False

        # Open the port if not already opened
        if not self.serialPort.is_open:
            self.serialPort.open()




    def CloseSerialPort(self):
        # Close the serial port if it is open
        if self.serialPort.is_open:
            self.serialPort.close()



    def WriteFES(self, bitString):

        # Write a bitstring through the open serial port
        if self.serialPort.is_open:

            # Convert the bit string in to a bytes object
            byteString = int(bitString, 2).to_bytes(len(bitString) // 8, byteorder='big')
            bytesWritten = self.serialPort.write(byteString)
            self.serialPort.flush()

        else:
            print("Error writing to FES device: serial port not opened")



    # A method to initialize the device into Channel List Mode
    def InitializeChannelListMode(self):

        # Set ts1 and ts2
        ts1 = 1000  / self.stimulationFrequency
        ts2 = 1000 / self.groupStimulationFrequency

        # Compute the group and main times
        mainTime = int((ts1 - 1.0) / 0.5)
        groupTime = int((ts2 - 1.5) / 0.5)

        # Set the active channels according to the input
        activeChannelsString = "11111111"

        # Specify which channels apply the scaler to get lower frequencies (none by default)
        lowFrequencyChannelsString = "00000000"

        # Compute checksum
        checksum = (self.nFactor + int(activeChannelsString) + int(lowFrequencyChannelsString) + groupTime + mainTime) % 8

        # Convert each parameter to its correct binary representation
        mainTimeBinary = "{0:011b}".format(mainTime)                              # binary, 11 bit width, 0 padding
        nFactorBinary = "{0:03b}".format(self.nFactor)                                 # binary, 3 bit width, 0 padding
        groupTimeBinary = "{0:05b}".format(groupTime)                             # binary, 5 bit width, 0 padding
        checksumBinary = "{0:03b}".format(checksum)                               # binary, 3 bit width, 0 padding

        # Compose the complete bistring
        bitString = "100"\
                    + checksumBinary\
                    + nFactorBinary[0:2]\
                    + "0"\
                    + nFactorBinary[2:3]\
                    + activeChannelsString[0:6]\
                    + "0"\
                    + activeChannelsString[6:8]\
                    + lowFrequencyChannelsString[0:5]\
                    + "0"\
                    + lowFrequencyChannelsString[5:8]\
                    + "00"\
                    + groupTimeBinary[0:2]\
                    + "0"\
                    + groupTimeBinary[2:5]\
                    + mainTimeBinary[0:4]\
                    + "0"\
                    + mainTimeBinary[4:11]

        # Write the bit string
        self.WriteFES(bitString)



    def UpdateChannelSettings(self, newAmplitudes, newPulses, pulseModes):

        # Ensure that the device amplitudes are capped between the safety bounds 0 - 50 mA
        for index, amplitude in enumerate(newAmplitudes):

            if amplitude > 50:

                self.amplitudes[index] = 50

            elif amplitude < 0:

                self.amplitudes[index] = 0

            else:

                self.amplitudes[index] = newAmplitudes[index]
                
        for i, pulse in enumerate(newPulses):
            self.pulseWidths[i] = pulse
        
        
        for i, mode in enumerate(pulseModes):
            self.pulseModes[i] = mode
        
            
        # Compute checksum
        checksum = (np.sum(self.pulseModes) + np.sum(self.pulseWidths) + np.sum(self.amplitudes)) % 32

        # Construct the channel settings string
        checksumBinary = "{0:05b}".format(checksum)

        channelSettingsBinary = ""
        for amplitude, pulseWidth, pulseMode in zip(self.amplitudes, self.pulseWidths, self.pulseModes):

            pulseWidthBinary = "{0:09b}".format(pulseWidth)
            channelSettingBinary = "0"\
                                   + "{0:02b}".format(pulseMode)\
                                   + "000"\
                                   + pulseWidthBinary[0:2]\
                                   + "0"\
                                   + pulseWidthBinary[2:9]\
                                   + "0"\
                                   + "{0:07b}".format(amplitude)

            channelSettingsBinary += channelSettingBinary

        # Compose the final bitstring
        bitString = "101" + checksumBinary + channelSettingsBinary

        # Write the bit string
        self.WriteFES(bitString)


    def StopDevice(self):

        # Compose the stop command
        stopCommandBinary = "11000000"

        # Write the bit string
        self.WriteFES(stopCommandBinary)

# Процедура первоначального картирования
1. Поиск срединного нерва. 
- Используйте немного проводящего геля
- Сгибание разгибание кисти помогает варьировать ощущения
- Устанавливайте комфортную амплитуду, так как во время проведения эксперимента будут предъявляться стимулы с большей шириной, а следовательно и интенсивностью

<img src="resources/median_nerve.jpg">

2. Просим пациента изучить содержание опросника. Отвечаем на появившиеся вопросы и поясняем термины.
3. Варьируем ширину, частоту и длительность. Просто запускаем ячейку c экспериментом.
4. Просим пациента заполнить опросник, отмечая зоны ощещений и описывающие слова
5. Анализ данных


In [ ]:
# Start MotionStim8 connection
try:
    mm=Motionstim8(freq=10)
    mm.OpenSerialPort('COM3') #COM5 on Misha's laptop
    mm.InitializeChannelListMode()
except:
    raise Exception('Unable to open serial port for MotionStim8 connection!\nTry to check MotionStim8 port or reboot kernel.')

print('Set critical amplitude')
crit_amp = int(input())
    
while True:
    amplitude = input()
    if amplitude == 'n':
        print('Stop')
        break
    else:
        amplitude = int(amplitude)
        
    if amplitude <= crit_amp:
        mm.UpdateChannelSettings(newAmplitudes=[amplitude], 
                                 newPulses=[150], 
                                 pulseModes=[0])
    else:
        print(f'Too high stimulation amplitude')
        continue

In [3]:
pulses = [50, 100, 200, 400]
freqs = [8, 15, 30, 50, 95]
durations = [50, 250, 500]
combinations = list(product(pulses, freqs, durations))
combinations = random.sample(combinations, len(combinations))
wrong_pool = '? \ | / : * < >'.split(' ') # To check whether name was written correctly
# To not face problems with 
name_csv = input(f"Enter subject's ID:\t") # To correctly asess log files and csvs if needed
flag = True
while(flag):
    flag = False
    for i in wrong_pool:
        if i in name_csv:
            print('Wrong subject name\t Enter again:\t')
            name_csv = input(f"Enter subject's ID:\t")
            flag = True

newpath = f'data/{name_csv}'
if not os.path.exists(newpath): {os.makedirs(newpath)}
log_file = open(f'{newpath}/{name_csv}_log.txt', 'w')

Enter subject's ID:	tr


In [5]:
# Write-to-csv function
def csv_save(dataF, naming_file):
    
    pd.DataFrame(dataF).to_csv(f'{newpath}/{naming_file}.csv', index=False)

#### Stimulation
amp = 6
picture = 0
data = {'Number': [], 'Date': [], 'Time': [], 'Pulse': [],
        'Frequency': [], 'Duration': [], 'Value': [], 'Amp': [], 'Picture': []}
# name_csv = input(f"Enter subject's ID:\t")

for i, (pulse, freq, duration) in enumerate(combinations):
    cur_time=time.strftime("%b-%d %H_%M_%S", time.gmtime())
    
    if 1000 / freq > duration:
        print(f'Pic: "", Num: {i + 1}, Skip combination: {pulse, freq, duration, cur_time}')
        log_file.write(f'{i + 1}, Skipped {cur_time}, pulse: {pulse}mcs, freq: {freq}Hz, duration: {duration}ms , picture: no\n')
        
        timing = cur_time.split(' ')
#         if i == 0:
#             name_csv += (str(timing[0]) + '_' + str(timing[1]) + '-to-')
        data['Number'].append(i+1)
        data['Date'].append(timing[0])
        data['Time'].append(timing[1])
        data['Pulse'].append(pulse)
        data['Frequency'].append(freq)
        data['Duration'].append(duration)
        data['Value'].append('Skipped')
        data['Amp'].append(amp)
        data['Picture'].append('')
        continue
    
    picture = input()
    if picture == 'n':
        log_file.close()
        #name_csv += str(timing[1])
        csv_save(data, name_csv)
    
        break
    
    cur_time=time.strftime("%b-%d %H_%M_%S", time.gmtime())    
    print(f'Pic: {picture}, Num: {i + 1}, {cur_time}, pulse: {pulse}mcs, freq: {freq}Hz, duration: {duration}ms')
    log_file.write(f'{i + 1}, {cur_time}, pulse: {pulse}mcs, freq: {freq}Hz, duration: {duration}ms , picture: {picture}\n')

    timing = cur_time.split(' ')
#     if i == 0:
#         name_csv += (str(timing[0]) + '---' + str(timing[1]) + '-to-')
    data['Number'].append(i+1)
    data['Date'].append(timing[0])
    data['Time'].append(timing[1])
    data['Pulse'].append(pulse)
    data['Frequency'].append(freq)
    data['Duration'].append(duration)
    data['Value'].append('Passed')
    data['Amp'].append(amp)
    data['Picture'].append(picture)
    
    mm.stimulationFrequency = freq
    mm.InitializeChannelListMode()
    time.sleep(0.1)
    mm.UpdateChannelSettings(newAmplitudes=[amp], newPulses=[pulse], pulseModes=[0])
    time.sleep(duration / 1000)
    mm.UpdateChannelSettings(newAmplitudes=[0], newPulses=[pulse], pulseModes=[0])
else:
    #name_csv += str(timing[1])
    csv_save(data, name_csv)
    log_file.close()
    


Pic: "", Num: 1, Skip combination: (50, 15, 50, 'Mar-07 20_32_59')
Skipped
1
Pic: 1, Num: 2, Mar-07 20_33_01, pulse: 50mcs, freq: 8Hz, duration: 250ms
Passed
Pic: "", Num: 3, Skip combination: (50, 8, 50, 'Mar-07 20_33_01')
Skipped
2
Pic: 2, Num: 4, Mar-07 20_33_02, pulse: 100mcs, freq: 15Hz, duration: 250ms
Passed
3
Pic: 3, Num: 5, Mar-07 20_33_03, pulse: 100mcs, freq: 8Hz, duration: 250ms
Passed
Pic: "", Num: 6, Skip combination: (100, 15, 50, 'Mar-07 20_33_04')
Skipped
Pic: "", Num: 7, Skip combination: (100, 8, 50, 'Mar-07 20_33_04')
Skipped
4
Pic: 4, Num: 8, Mar-07 20_33_05, pulse: 50mcs, freq: 15Hz, duration: 250ms
Passed
